# Word2Vec con Gensim

## Instalación de librerías y carga de dataset

In [ ]:
#poetry add datasets

In [2]:
from gensim.models import Word2Vec
import pandas as pd
import re
from gensim.parsing.preprocessing import strip_punctuation, strip_numeric, strip_short, stem_text
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

-------------------------

In [3]:
from datasets import load_dataset

dataset_corpus = load_dataset("large_spanish_corpus", "ParaCrawl")

Generating train split:   0%|          | 0/15510649 [00:00<?, ? examples/s]

In [5]:
subset = dataset_corpus['train'].select(range(1000000))

In [6]:
subset[0:2]

{'text': ['lavado de cerebro a través de los medios de comunicación, y amenaza de fuerza a través de los militares.',
  'Sin un constante aluvión de doble cañón, requiriendo la complicidad de los seres humanos para reprimir y engañar a sus semejantes, su tan cacareada magia rápidamente se desvanecería y se disiparía.']}

In [7]:
import nltk
nltk.download("stopwords")
nltk.download("punkt")
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nestorxyz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/nestorxyz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Procesamiento de texto

In [8]:
STOP_WORDS = set(stopwords.words('spanish'))

def clean_text(sentence_batch):
    # extrae el texto de la entrada
    test_list = sentence_batch["text"]

    cleaned_text_list = []

    for text in test_list:
        # Convierte el texto a minúsculas
        text = text.lower()

        # Elimina URLs
        text = re.sub(r'https?://\S+|www\.\S+', '', text, flags=re.MULTILINE) # flags es para que no se queje de que no encuentra el patrón

        # Elimina las menciones @ y '#' de las redes sociales
        text = re.sub(r'\@\w+|\#','', text)

        # Elimina los caracteres de puntuación
        text = strip_punctuation(text)

        # Elimina los números
        text = strip_numeric(text)

        # Elimina las palabras cortas
        text = strip_short(text, minsize=2)

        # Elimina las palabras comunes (stop words)
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word not in STOP_WORDS]

        cleaned_text_list.append(filtered_text)

    # Devuelve el texto limpio
    return {"text": cleaned_text_list}



In [9]:
sentence_corpus = subset.map(clean_text, batched=True)

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [10]:
sentence_corpus[0:2]

{'text': [['lavado',
   'cerebro',
   'través',
   'medios',
   'comunicación',
   'amenaza',
   'fuerza',
   'través',
   'militares'],
  ['constante',
   'aluvión',
   'doble',
   'cañón',
   'requiriendo',
   'complicidad',
   'seres',
   'humanos',
   'reprimir',
   'engañar',
   'semejantes',
   'tan',
   'cacareada',
   'magia',
   'rápidamente',
   'desvanecería',
   'disiparía']]}

## Carga y uso de modelo de embeddings Word2Vec

In [11]:
model = Word2Vec(sentences=sentence_corpus["text"], vector_size=100, window=5, min_count=2, workers=4, sg=1)

# Podemos guardar el modelo para uso futuro
model.save("word2vec.model")

In [15]:
model.wv.most_similar("comida",topn=10)

[('comer', 0.7817370891571045),
 ('comidas', 0.7424415946006775),
 ('sabrosa', 0.7298787236213684),
 ('cocinar', 0.7235280275344849),
 ('sopa', 0.7228937149047852),
 ('desayunar', 0.7153236865997314),
 ('bebida', 0.7118685841560364),
 ('deliciosa', 0.7118524312973022),
 ('bocadillos', 0.7072049975395203),
 ('merienda', 0.7068880200386047)]

In [16]:
##comida, ser, reina, television
word_vectors = model.wv
vectors = word_vectors.vectors
words = word_vectors.index_to_key

## Almacenamiento de embeddings

In [ ]:
df_vectors = pd.DataFrame(vectors, index=words)
df_vectors.to_csv("word2vec.csv", sep="\t", index=False)